In [1]:
import csv
import json
import pandas as pd
import sys, getopt, pprint
import twitter
import tweepy
import time
import collections 
import datetime 
import re
import pymongo
from bson import json_util, ObjectId
import numpy as np
import requests
import configparser
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import warnings
import random
import pprint
import pandas as pd
import MySQLdb
import sqlalchemy
import pprint
import os
import bson

In [ ]:
#MIT License

#Copyright <YEAR> <COPYRIGHT HOLDER>

#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [3]:
#Application authoration information
CONSUMER_KEY = 'kPDlBul6nS3moLfGmLvwM3pLm'
CONSUMER_SECRET = 'EQ8nv28WyALnhZ8jehwGlLepMB7eBzKDOZOggnIsqukWcIFX1A'
OAUTH_TOKEN = '3910298715-SQR4mBRJe1G59wybNQbo1NvCFneJRuIqd3D18CB'
OAUTH_TOKEN_SECRET = '86pVFTMoquM1h753aPvNV6GHcEuu5FUD6URn5pZA90xiJ'

In [4]:
#Building connction with tweepy
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=3, retry_delay=60)

In [8]:
#Create empty list for data extraction
t = []

In [14]:
#Data extraction using tweepy api and store data in empty list. 
for tweet in tweepy.Cursor(api.search, q=('monster hunter world'), since='2018-04-19', until='2018-04-20').items(100):
    #tweet_id = tweet.id
    t += [{ 'Tweet_id': tweet.id,
            'Game_id': '488',
            'Screen_name':tweet.author.screen_name.encode('utf8'),
            'Created_at':tweet.created_at,
            'Tweet_text':tweet.text.encode('utf8'),
            'Hashtags':re.findall(r"#(\w+)",tweet.text),
            'Retweets':tweet.retweet_count,
            'Favorites':tweet.favorite_count,
            'Location':tweet.user.location.encode('utf8')}]

In [15]:
#Convert list into pandas dataframe. 
df1 = pd.DataFrame(t)

In [9]:
#Self define functions for data cleaning. 
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

def split_df(dataframe, col_name, sep):
    orig_col_index = dataframe.columns.tolist().index(col_name)
    orig_index_name = dataframe.index.name
    orig_columns = dataframe.columns
    dataframe = dataframe.reset_index()  # we need a natural 0-based index for proper merge
    index_col_name = (set(dataframe.columns) - set(orig_columns)).pop()
    df_split = pd.DataFrame(
        pd.DataFrame(dataframe[col_name].str.split(sep).tolist())
        .stack().reset_index(level=1, drop=1), columns=[col_name])
    df = dataframe.drop(col_name, axis=1)
    df = pd.merge(df, df_split, left_index=True, right_index=True, how='inner')
    df = df.set_index(index_col_name)
    df.index.name = orig_index_name
    # merge adds the column to the last place, so we need to move it back
    return change_column_order(df, col_name, orig_col_index)

In [17]:
#Data cleaning procedure.
df1['Hashtags'] = df1['Hashtags'].astype('str')
df1['Created_at'] = df1['Created_at'].astype('str')
df1['Screen_name'] = df1['Screen_name'].astype('str')
df1['Tweet_text'] = df1['Tweet_text'].astype('str')
df1['Tweet_id']= df1['Tweet_id'].astype('str')
df1['Location']= df1['Location'].astype('str')

In [18]:
#Data cleaning procedure cont. 
df1['Hashtags'] = df1['Hashtags'].str.replace('[','')
df1['Hashtags'] = df1['Hashtags'].str.replace(']','')

In [19]:
#Data cleaning procedure cont. 
df1 = split_df(df1, 'Hashtags', ', ')

In [21]:
#Data cleaning procedure cont. 
df1['Screen_name']=df1['Screen_name'].str[2:-1]
df1['Tweet_text'] = df1['Tweet_text'].str[2:-1]
df1['Location']=df1['Location'].str[2:-1]
df1['Tweet_id1']=df1['Tweet_id'].str[:9]
df1['Tweet_id2']=df1['Tweet_id'].str[9:]

In [22]:
#Data cleaning procedure cont. 
df1 = df1.drop(['Tweet_id'],axis=1)

In [23]:
#Data cleaning procedure cont. 
df1['Hashtags']=df1['Hashtags'].str[1:-1]

In [24]:
#Data cleaning procedure cont. 
df1['Tweet_date']=df1['Created_at'].str[:10]
df1['Tweet_time']=df1['Created_at'].str[10:]

In [25]:
#Data cleaning procedure cont. 
df1 = df1.drop(['Created_at'],axis=1)

In [2]:
#Enter the values for database connection
dsn_database = "project"   # e.g. "MySQLdbtest"
dsn_hostname = "127.0.0.1"       # e.g.: "mydbinstance.xyz.us-east-1.rds.amazonaws.com"
dsn_port = 3306                        # e.g. 3306 
dsn_uid = "root"             # e.g. "user1"
dsn_pwd = "12345"            # e.g. "Password123"

In [3]:
#sqlalchemy package creates connection with mysql database to import dataframe into MySQL.
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(dsn_uid, dsn_pwd, 
                                                      dsn_hostname, dsn_database))

In [28]:
#Importing file into MySQL. 
df1.to_sql(con=database_connection, name='tweets', if_exists='append',index=False)

In [3]:
#Create connection with MySQL using MySQLdb to perform selection. 
conn = MySQLdb.connect(host=dsn_hostname, port=dsn_port, user=dsn_uid, passwd=dsn_pwd, db=dsn_database)
cursor=conn.cursor()

In [7]:
#1. What are people saying about me (somebody)?
# Select all hashtags included in all tweets about this game. 
cursor.execute("""select Hashtags from Tweets where Hashtags is not null GROUP BY Hashtags limit 10""")
rows = cursor.fetchall()
pprint.pprint(rows)

(('02',),
 ('1',),
 ('14',),
 ('2',),
 ('3',),
 ('5',),
 ('5060',),
 ('50K',),
 ('5739',),
 ('BonPlan',))


In [9]:
#2. How viral are my posts?
# Order tweets by retweet numbers. 
cursor.execute("""select Retweets, Tweet_id1, Tweet_id2 from Tweets where Retweets != 0 group by Tweet_text order by Retweets DESC limit 10""")
rows = cursor.fetchall()
pprint.pprint(rows)

((20095, 986019774, 399434753),
 (7447, 986393866, 877087746),
 (1356, 986393207, 306051584),
 (1086, 985658666, 593472513),
 (912, 986393139, 370799104),
 (464, 986392164, 769976320),
 (459, 986755483, 590537216),
 (407, 986393803, 433979904),
 (311, 986746988, 682084353),
 (213, 986393685, 737787392))


In [10]:
#3. How much influence to my posts have?
# Order tweets by favorite number 
cursor.execute("""select Favorites, Retweets, Tweet_id1, Tweet_id2 from tweets where Retweets !=0 and Favorites !=0 group by Tweet_text order by Favorites DESC limit 10""")
rows = cursor.fetchall()
pprint.pprint(rows)

((44, 2, 987103597, 262442496),
 (16, 3, 986749897, 717231621),
 (9, 8, 986020388, 462383107),
 (8, 4, 985668186, 518335488),
 (8, 2, 985650733, 797257216),
 (7, 3, 985664925, 761613824),
 (6, 1, 985661712, 736116742),
 (5, 2, 985665132, 226334720),
 (5, 1, 986024159, 70343173),
 (5, 10, 986016914, 43420672))


In [11]:
#4. What posts are like mine?
# Select tweets with same hashtag. 
cursor.execute("""select tweet_id1,tweet_id2 from tweets where Hashtags='PS4live' limit 10""")
rows = cursor.fetchall()
pprint.pprint(rows)

((985666520, 75218944),
 (985666264, 348659714),
 (985665957, 464039424),
 (985665458, 459291648),
 (985663785, 808662528),
 (985660602, 34122752),
 (985656642, 44805120),
 (985655896, 897478657),
 (985655059, 34845185),
 (985653975, 994241024))


In [12]:
#5. What users post like me?
#Select screen_name used the same hashtags in their tweets. 
cursor.execute("""select distinct Screen_name from tweets where Hashtags = 'PS4live' limit 10""")
rows = cursor.fetchall()
pprint.pprint(rows)

(('TM_SweetTooth',),
 ('raceryayo',),
 ('Hphsbillz',),
 ('JF_Chabot',),
 ('Tr11Royalty',),
 ('Papichulo_KD',),
 ('KennyJay305',),
 ('EP1C_K1LLSW1TCH',),
 ('mdawggaming',),
 ('FetterlyTravis',))


In [13]:
#6. Who should I be following?
#Select users who post about this topic the most. 
cursor.execute("""select Screen_name, count(*) from tweets group by Screen_name order by count(*) DESC limit 10;""")
rows = cursor.fetchall()
pprint.pprint(rows)

(('dealPS4', 5),
 ('MissMesho', 5),
 ('GermanTwitchRT', 4),
 ('StreamAcademyRT', 4),
 ('Death280', 4),
 ('ADSPLAY101', 4),
 ('pino_4521', 4),
 ('deals_vtipha', 3),
 ('MrJoeyOverdose', 3),
 ('AngeloOhhBaby', 3))


In [17]:
#7. What topics are trending in my domain?
# Select most used hashtags in a certain date to represent the trends. 
cursor.execute("""select Hashtags,count(Hashtags)from tweets where Hashtags is not null and Tweet_date='2018-04-19' group by Hashtags order by count(Hashtags) DESC limit 3""")
rows = cursor.fetchall()
pprint.pprint(rows)

(('PS4live', 15), ('Twitch', 3), ('Streaming', 3))


In [18]:
#8. What keywords hashtags should I add to my post?
# Select the most used hashtags. 
cursor.execute("""select Hashtags,count(Hashtags) from tweets where Hashtags is not null group by Hashtags order by count(Hashtags) DESC limit 3;""")
rows = cursor.fetchall()
pprint.pprint(rows)

(('MHWorld', 54), ('PS4live', 45), ('MonsterHunterWorld', 9))


In [19]:
#9. Should I follow somebody back?
#Select the top 3 users posting about this topic. 
cursor.execute("""select Screen_name, count(*) from tweets group by Screen_name order by count(*) DESC limit 3;""")
rows = cursor.fetchall()
pprint.pprint(rows)

(('dealPS4', 5), ('MissMesho', 5), ('GermanTwitchRT', 4))


In [4]:
#10. What is the best time to post?
#Select time point with most tweets occured. 
cursor.execute("""select Tweet_time, count(Tweet_time) from tweets group by Tweet_time order by count(Tweet_time) DESC Limit 3;""")
rows = cursor.fetchall()
pprint.pprint(rows)

(('23:59:58', 5), ('23:40:25', 5), ('23:57:21', 4))


In [6]:
#13. What’s my reach?
# From Japan to the location of all the tweets. 
cursor.execute("""select distinct Location from tweets where Location is not null limit 10""")
rows = cursor.fetchall()
pprint.pprint(rows)

(('#GamingIsInOurBlood',),
 ('40.868181,-73.963332',),
 ('A Computer Somewhere',),
 ('a hole',),
 ('AB, Canada',),
 ('Alabama, USA',),
 ('Algiers                       ',),
 ('Alternative Reality, USA',),
 ('Amiens, France',),
 ('Amsterdam, Nederland',))
